In [ ]:
from google.colab import drive
drive.mount('/content/drive')

/content/drive/MyDrive/Colab Notebooks/Week#3/MiniProject/RiceTyoe.zip 에서 unzip

In [19]:
!mkdir RiceType
!!unzip /content/drive/MyDrive/Colab_Files/Week_3/MiniProject/RiceTyoe.zip -d /content/RiceType

mkdir: cannot create directory ‘RiceType’: File exists


['unzip:  cannot find or open /content/drive/MyDrive/Colab_Files/Week_3/MiniProject/RiceTyoe.zip, /content/drive/MyDrive/Colab_Files/Week_3/MiniProject/RiceTyoe.zip.zip or /content/drive/MyDrive/Colab_Files/Week_3/MiniProject/RiceTyoe.zip.ZIP.']

관련 library import.
서론 : 데이터, 설명
본론 : 적용한 방법론 -> 데이터 전처리, 활용한 AutoML 라이브러리, 실험셋팅 등
                     -> 실험결과 설명 : AutoML 결과 비교등
결론 : AutoML의 장단점, 개선방향및 의견

In [20]:
import os
import time
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.model_selection
import warnings
import logging
from warnings import simplefilter

!pip install auto-sklearn==0.10.0

warnings.filterwarnings('ignore')
logging.captureWarnings(True)
simplefilter(action='ignore', category=FutureWarning)

Autosklearn Classification?

In [ ]:
import autosklearn.classification

Origin_Data= pd.read_csv('/content/RiceType/riceClassification.csv')
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split()
print(f'Sizes: X_train={X_train.shape}, y_train={y_train.shape}, X_test={X_test.shape}, y_test={y_test.shape}')


AutoGluon으로 한번 시작해보자.

In [ ]:
!pip install --upgrade pip --user
!pip install "mxnet_cu101<2.0.0, >=1.7.0"
!pip install autogluon # 
from autogluon.tabular import TabularDataset, TabularPredictor
# AutoML에서 CASH(Combined Algorithm Selection and Hyperparameter optimization problem)는 최적의 기계 학습 모델(algorithm)
#  및 해당 하이퍼 파라미터(hyperparameter)를 탐색하는 작업을 말합니다. 이 검색 공간은 대부분의 경우 비 연속적이기 때문에 계산 비용이 매우 높아진다는 특징이 있습니다. 
# 예를 들어, 잘 알려진 하이퍼 매개 변수 탐색 방법으로 그리드 검색이 있지만, 이것은 모든 하이퍼 매개 변수의 조합을 전부 조사하는 방법으로 매우 비효율적입니다.

# 기존 AutoML 프레임워크와 달리 AutoGluon-Tabler는 CASH를 해결하는 것을 중요시하지 않습니다.
#  무슨 말인가 하면 앞서 본 바와 같이 CASH란 모델과 하이퍼 매개 변수의 탐색 작업이며, 모델에 입력하는 데이터의 전처리는 분석가가 직접 수행해야 합니다.
#   그러나 실제 기계 학습에서 가장 귀찮은 것이 이 부분입니다. 
# AutoGluon-Tabuler는 고급 데이터 처리를 통해 이러한 문제를 해결하면서 신경망과 계층 구조를 가진 앙상블 기법을 통해 테이블 데이터의 학습에서 높은 성능을 실현하고 있습니다.

데이터 넣어주기

In [ ]:
train_data= pd.read_csv('/content/RiceType/riceClassification.csv')
# subsample subset of data for faster demo, try setting this to much larger values 
test_num = 5000
X_train = train_data[:test_num]
X_test = train_data[test_num:]
# y_train= X_train['Class']
# y_test = X_test['Class']

train_data.head()
# class jasmine -> 1 Gonen -> 0 
print(train_data)

In [ ]:
sns.heatmap(train_data.isnull(), cbar=False) # null 값 없음을 확인

In [ ]:
label = 'Class'
print("Summary of Class Variable =\n",train_data[label].describe())

In [ ]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

In [ ]:
y_test = X_test[label]  # values to predict
test_data_nolab = X_test.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

In [ ]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [ ]:
predictor.leaderboard(X_test, silent=True)

In [ ]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5) 
# 앞부분의 확률을 분석하여 앞 5행에 대한 확률을 산출해냈음

In [ ]:
results = predictor.fit_summary()

In [ ]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

In [ ]:
predictor.predict(X_test, model='LightGBM')

Maximixing Predictive Performance


In [ ]:
time_limit = 120  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')
predictor.leaderboard(X_test, silent=True)

In [ ]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

In [ ]:
predictor_age = TabularPredictor(label='Class', path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(X_test)

In [ ]:
predictor_age.leaderboard(test_data, silent=True)